# Seq2Seq 모델 구현 및 챗봇데이터 학습


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [3]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [4]:
!bash install_mecab-ko_on_colab_light_210108.sh

Installing konlpy.....
     |████████████████████████████████| 19.4 MB 57.9 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 65.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-09-30 14:12:14--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::34cc:ea4a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=TItyuRC8K2BLauaQBQ2HG5nhN40%3D&

In [5]:
cd ..

/content


In [6]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from tqdm.auto import tqdm


import pandas as pd

from konlpy.tag import Mecab


class LSTMEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, embedding_dim, num_layers=1 ,batch_first=True)

    def forward(self, encoder_embeds):
        hidden_and_cell = self.lstm(encoder_embeds)[1]
        return hidden_and_cell

    
class LSTMDecoder(nn.Module):
    def __init__(self, embedding_dim):
        super(LSTMDecoder, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, embedding_dim, num_layers=1 ,batch_first=True)

    def forward(self, decoder_embeds, hidden_and_cell):
        decoder_output = self.lstm(decoder_embeds, hidden_and_cell)[0]
        return decoder_output


class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Seq2Seq, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = LSTMEncoder(embedding_dim)
        self.decoder = LSTMDecoder(embedding_dim)
        self.vocab_proj = nn.Linear(embedding_dim, vocab_size)

    def forward(self, encode_input, decode_input):
        encoder_embeds = self.word_embedding(encode_input)
        decoder_embeds = self.word_embedding(decode_input)
        
        hidden_and_cell = self.encoder(encoder_embeds)
        decoder_output = self.decoder(decoder_embeds, hidden_and_cell)
        projected_output = self.vocab_proj(decoder_output)
        
        return projected_output


In [7]:
chatbot_data = pd.read_csv('/content/drive/MyDrive/chat-with-U/chatbot-study/seq2seq/chatbot_data/ChatbotData.csv')
chatbot_data

#형태소 분석기 호출 -> 형태소 별로 단어를 나누고 이를 set으로 만들어 vocab생성
pos_tagger = Mecab()

question = chatbot_data.Q # Seq2Seq에 encoder_input
answer = chatbot_data.A # Seq2Seq에 decoder_input
total_utterances = pd.concat((question,answer)) #vocab을 만들기위한 전체 발화문장

단어 임베딩 vocab을 만들기위해 아래와같이 단어별로 나눈 뒤 vocab set에 넣어줍니다.

set에 add되기때문에 중복되는 vocab은 존재하지 않습니다.

In [8]:
vocab = []

special_tokens = ['[PAD]', '[MASK]', '[START]', '[END]', '[UNK]']
for special_token in special_tokens:
    vocab.append(special_token)

for utterance in total_utterances:
    for eojeols in pos_tagger.pos(utterance,flatten=False, join=True):
        count = 0
        for token in eojeols:
            if count > 0:
                if token in vocab:
                    continue
                vocab.append('##' + token)
            else:
                if token in vocab:
                    continue
                vocab.append(token)
                count += 1
        
vocab_size = len(vocab)

In [9]:
token2index = {token : index for index, token in enumerate(vocab)}
index2token = {index : token for index, token in enumerate(vocab)}

In [10]:
vocab_size = len(vocab)

In [11]:
class WordHandler:
    def __init__(self, vocab, pos_tagger, token2index, index2token):
        self.vocab = vocab
        self.pos_tagger = pos_tagger
        self.token2index = token2index
        self.index2token = index2token
        
    def encode(self, sentence):
        encoded_vector = [self.token2index[token] if token in self.token2index else self.token2index['[UNK]']
                          for token in self.pos_tagger.pos(sentence, join= True)]
        
        return encoded_vector
    
    def decode(self, indice, join=True):
        decoded_vector = [self.index2token[index] for index in indice]
        
        return decoded_vector
    
    def decode_without_tag(self, indice):
        decoded_vector = ' '.join([self.index2token[index].split('/')[0] for index in indice])
        
        return decoded_vector
    
    @staticmethod
    def return_max_seq_len(sentences):
        max_seq_len = 0
        for sentence in sentences:
            max_seq_len = max(len(sentence), max_seq_len)
        
        return max_seq_len
            
        
        

In [50]:
class ChitChatDataset(Dataset):
    def __init__(self, input_ids, output_ids, index2token, token2index, max_seq_len):
        self.input_ids = input_ids
        self.output_ids = output_ids
        self.index2token = index2token
        self.token2index = token2index
        self.max_seq_len = max_seq_len

    def __getitem__(self, idx):
        
        if len(self.input_ids[idx]) < self.max_seq_len:
            padding_block = self.max_seq_len - len(self.input_ids[idx]) + 2
            encode_input = torch.LongTensor(
                            self.input_ids[idx] + 
                            [self.token2index['[PAD]']] * padding_block
                            )
        else:
            encode_input = torch.LongTensor(
                            self.input_ids[idx]
                            )
        
        if len(self.output_ids[idx]) + 1 < self.max_seq_len:
            padding_block = self.max_seq_len - len(self.output_ids[idx]) + 2
            decode_input = torch.LongTensor( 
                            [self.token2index['[START]']] + 
                            self.output_ids[idx] + 
                            [self.token2index['[PAD]']] * padding_block
                            )
            target = torch.LongTensor( 
                            self.output_ids[idx] + 
                            [self.token2index['[END]']] + 
                            [self.token2index['[PAD]']] * padding_block
                            )
        else:
            decode_input = torch.LongTensor( 
                            [self.token2index['[START]']] + 
                            self.output_ids[idx]
                            )
            target = torch.LongTensor( 
                            self.output_ids[idx] + 
                            [self.token2index['[END]']]
                            )

        return encode_input, decode_input, target

    def __len__(self):
        return len(self.input_ids)

In [51]:
handler = WordHandler(vocab, pos_tagger, token2index, index2token)

In [52]:
input_ids = question.map(handler.encode)
output_ids = answer.map(handler.encode)

In [53]:
max_seq_len = max(handler.return_max_seq_len(output_ids), handler.return_max_seq_len(input_ids))
max_seq_len

40

In [56]:
chitchat_data = ChitChatDataset(input_ids, output_ids, index2token, token2index, 50)
chitchat_dataloader = DataLoader(chitchat_data, batch_size= 100, shuffle = True)

In [57]:
model = Seq2Seq(vocab_size, 512).cuda()

criterion = nn.CrossEntropyLoss()
lr = 0.0001
optimizer = Adam(model.parameters(), lr)
step = 0



for epoch in tqdm(range(100)):
    for encode_input, decode_input, target in tqdm(chitchat_dataloader):
        encode_input = encode_input.cuda()
        decode_input = decode_input.cuda()
        target = target.cuda()
        step += 1
        optimizer.zero_grad()
        output = model(encode_input, decode_input)
        
        loss = 0
        for i in range(output.size()[0]):
            loss += criterion(output[i][:len(target[i])], target[i])
            
        loss.backward()
        optimizer.step()
        

    sample_input = handler.decode_without_tag(encode_input[0].tolist())
    sample_output = handler.decode_without_tag(output[0].argmax(-1).tolist())
    print(f'loss: {loss}')
    print(f'input_sentence: {sample_input}')
    print(f'output_sentence: {sample_output}')

        
        
        

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

loss: 21.57735252380371
input_sentence: 썸남 이랑 뭐 하 지 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
output_sentence: [PAD] . . . . . . . . [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


  0%|          | 0/119 [00:00<?, ?it/s]

loss: 19.74547576904297
input_sentence: 잊어 볼 마음 에 다른 여자 만나 고 왔 습니다 . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
output_sentence: [UNK] 이 이 . . . . . . . [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


...

loss: 3.057447910308838
input_sentence: 소화 가 안 돼 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
output_sentence: 제 챙겨 드세요 . [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


  0%|          | 0/119 [00:00<?, ?it/s]

loss: 3.097146987915039
input_sentence: 예전 게임 인데 재밌 다 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
output_sentence: 잘 도 하 고 싶 네요 . [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


  0%|          | 0/119 [00:00<?, ?it/s]

loss: 3.5456228256225586
input_sentence: 사람 많 아서 기다려야 할지 도 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
output_sentence: 제 는 마시 며 기다려 보 세요 . [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
